This will help you get started with the Composio. Composio enables your AI agents and LLMs to connect with 100+ external applications and services through a unified interface. For detailed documentation of all features and configurations, head to the [Composio Docs](https://docs.composio.dev/).

## Overview
Composio is a comprehensive toolkit that provides:

- **100+ Integrations:** Connect to popular applications like GitHub, Gmail, Slack, HubSpot, Salesforce, and more
- **Universal API:** One interface to manage all your external app interactions
- **Authentication Management:** Handle OAuth, API keys, and other auth methods automatically
- **Execution Environment:** Secure sandboxed execution of actions across different platforms
- **Function Calling:** Native support for LLM function calling with proper schema validation

## Setup

This toolkit lives in the `composio_langchain` package:

In [1]:
%pip install -qU composio_langchain==0.8.0 langchain

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
composio-babyagi 0.5.40 requires composio_core<=0.5.40,>=0.5.30, which is not installed.
cdp-langchain 0.0.13 requires langchain-openai<0.3.0,>=0.2.4, but you have langchain-openai 0.3.28 which is incompatible.
composio-crewai 0.7.1 requires composio_langchain<0.8.0,>=0.5.0, but you have composio-langchain 0.8.0 which is incompatible.
composio-langgraph 0.5.35 requires composio-langchain<=0.5.35,>=0.5.30, but you have composio-langchain 0.8.0 which is incompatible.
embedchain 0.1.125 requires langchain-openai<0.3.0,>=0.2.1, but you have langchain-openai 0.3.28 which is incompatible.
embedchain 0.1.125 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.4.7 which is incompatible.
embedchain 0.1.125 requires tiktoken<0.8.0,>=0.7.0, but you have tiktoken 0.9.0 which is incompatible.
langchain-community 0.3.3 

You'll also need to set up your Composio API key. Get one from [platform.composio.dev](https://platform.composio.dev):

In [6]:
import os
import getpass

os.environ["COMPOSIO_API_KEY"] = getpass.getpass("Enter your Composio API key: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Instantiation

Now we can instantiate Composio:

In [2]:
from composio import Composio
from composio_langchain import LangchainProvider

composio = Composio(
    api_key=os.environ["COMPOSIO_API_KEY"], provider=LangchainProvider()
)

## Connect your Account


We will be building an AI agent that can fetch your emails. For that you will need to connect your gmail account through Composio. You can do this on the composio platform or you can run the following code snippet:

In [ ]:
auth_config = composio.auth_configs.create(
    toolkit="gmail",
    options={
        "type": "use_composio_managed_auth",
    },
)
print(auth_config.id)
user_id = "default"
connection_request = composio.connected_accounts.initiate(
    user_id=user_id,
    auth_config_id=auth_config.id,
    config={"auth_scheme": "OAUTH2"},  # type: ignore
)
print(f"Redirect URL: {connection_request.redirect_url}")
connected_account = connection_request.wait_for_connection()

if connected_account.status == "ACTIVE":
    print(
        f"Account connected successfully, Connected account ID: {connected_account.id}"
    )
else:
    print(f"Failed to connect account, status: {connected_account.status}")

## Invocation

View available tools:

In [4]:
tools_list = composio.tools.get(user_id="default", toolkits=["GMAIL"])

Here's a [list](https://docs.composio.dev/toolkits/introduction) of all the toolkits supported by Composio

## Use within an agent

In [7]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4.1")

model_with_tools = model.bind_tools(tools_list)

In [ ]:
example_query = "Fetch and summarize the latest emails in my inbox."

result = model_with_tools.invoke(example_query)

print(result)

## API reference

For detailed documentation of all Composio features and configurations head to the [API reference](https://docs.composio.dev/).